In [1]:
# import packages
import pandas as pd
import numpy as np
from random import sample
from tqdm import tqdm, trange

In [79]:
# define function

def adv_index(list_to_index, list_to_match):
    return [ind for ind, match in enumerate(list_to_index) if match in list_to_match]

def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [170]:
# generate a small data

data = [['U1','I1','V'], ['U1','I1','P'], ['U1','I3','V'], ['U1','I4','V'],
        ['U2','I2','V'], ['U2','I2','P'], ['U2','I1','V'],
        ['U3','I1','V'], ['U3','I1','P'], ['U3','I2','V'], ['U3','I4','V'], ['U3','I4','P'],
        ['U4','I2','V'], ['U4','I3','V'], ['U4','I3','P'], ['U4','I4','V'],
        ['U5','I1','V'], ['U5','I3','V'], ['U5','I2','V'], ['U5','I2','P']
        ]
data = pd.DataFrame(data, columns=['UserID', 'ItemID', 'Action'])

item_list = list(set(data.ItemID))
user_list = list(set(data.UserID))
item_list.sort()
user_list.sort()

In [179]:
# parameter setting
dim = 5
num_iter = 2500
omega = 1
rho = 1
lambda_u = 0.1
lambda_v = 0.1
lambda_b = 0.1
gamma = 0.008
num_u = len(user_list)
num_i = len(item_list)

In [180]:
# Calculate auxiliary-target correlation C for every user and each types of auxiliary action
# Here we only have one auxiliary action 'V' for 'View'
target_action = 'P'
auxiliary_action = ['V']
C_u = dict()
for u in set(data.UserID):
    C_u[u] = dict()
    I_t_u = set(data[(data.UserID == u) & (data.Action == target_action)].ItemID)
    # TODO filtered item set
    for X in auxiliary_action:
        I_a_u = set(data[(data.UserID == u) & (data.Action == X)].ItemID)

        C_u_at = len(I_t_u.intersection(I_a_u)) / len(I_t_u)
        C_u_ta = len(I_t_u.intersection(I_a_u)) / len(I_a_u)

        C_u_X = 2 * C_u_at * C_u_ta / (C_u_ta + C_u_at)
        C_u[u][X] = C_u_X

temp = pd.DataFrame.from_dict(C_u, orient='index')
# We have only one auxiliary action 'V'
temp['alpha'] = omega * rho * temp.V
alpha_u = temp
del temp
alpha_u.reset_index(inplace=True)
alpha_u.columns = ['UserID', 'V', 'alpha']

In [181]:
# generate item-set based on co-selection
S = dict()
item_set_bar = tqdm(item_list)
for i in item_set_bar:
    S[i] = set()
    U_i = set(data[data.ItemID == i].UserID)
    for j in item_list:
        U_j = set(data[data.ItemID == j].UserID)
        if len(U_i.intersection(U_j)) >= 2: S[i].add(j)


100%|██████████| 4/4 [00:00<00:00, 129.11it/s]


In [182]:
# initialization
# we include item bias term in the last row of V
# and set the last column in U to all-1 vector
np.random.seed(20200701)
U = np.random.normal(size=(num_u, dim + 1))
V = np.random.normal(size=(dim + 1, num_i))
U[:, -1] = 1
# estimation is U dot V
estimation = np.dot(U, V)

In [183]:
# begin iteration
with trange(num_iter) as t:
    for index in t:
        # Description will be displayed on the left
        t.set_description('ITER %i' % index)

        # Build u, I, J, K
        # uniformly sample a user from U
        u = sample(set(data.UserID), 1)[0]
        # uniformly sample a item i from I_u_t
        I_u_t = set(data[(data.UserID == u) & (data.Action == 'P')].ItemID)
        i = sample(I_u_t, 1)[0]
        # build I = I_u_t cap S_i
        I = I_u_t.intersection(S[i])
        # build J, since we only have one auxiliary action, we follow the uniform sampling
        I_u_oa = set(data[(data.UserID == u) & (data.Action == 'V')].ItemID) - I_u_t# TODO: optimize this
        j = sample(I_u_oa, 1)[0]
        J = I_u_oa.intersection(S[j])
        # build K
        I_u_n = set(data.ItemID) - I_u_t - I_u_oa
        k = '' if len(I_u_n) == 0 else sample(I_u_n, 1)[0]
        K = I_u_n.intersection(S[k]) if k != '' else set()

        # calculate intermediate variables
        # get specific alpha_u
        spec_alpha_u = alpha_u[alpha_u.UserID == u].alpha.values[0]
        u_index = user_list.index(u)
        U_u = U[u_index, :-1]
        # get r_hat_uIJ and r_hat_uJK
        r_hat_uI = np.average(estimation[u_index, adv_index(item_list, sorted(I))])
        r_hat_uJ = np.average(estimation[u_index, adv_index(item_list, sorted(J))])
        r_hat_uK = np.average(estimation[u_index, adv_index(item_list, sorted(K))])

        r_hat_uIJ = r_hat_uI - r_hat_uJ
        r_hat_uJK = r_hat_uJ - r_hat_uK
        # get V_bar_I, V_bar_J, V_bar_K
        V_bar_I = np.average(V[:-1, adv_index(item_list, sorted(I))], axis=1)
        V_bar_J = np.average(V[:-1, adv_index(item_list, sorted(J))], axis=1)
        V_bar_K = np.average(V[:-1, adv_index(item_list, sorted(K))], axis=1)
        # get b_I, b_J, b_K
        b_I = np.average(V[-1, adv_index(item_list, sorted(I))])
        b_J = np.average(V[-1, adv_index(item_list, sorted(J))])
        b_K = np.average(V[-1, adv_index(item_list, sorted(K))])

        # calculate loss
        f_Theta = np.log(sigmoid(r_hat_uIJ / spec_alpha_u)) + np.log(sigmoid(r_hat_uJK))
        regula = lambda_u * np.linalg.norm(U_u, ord=2) + lambda_v * (np.linalg.norm(V_bar_I, ord=2) + np.linalg.norm(V_bar_J, ord=2) + np.linalg.norm(V_bar_K, ord=2)) + lambda_b * (b_I ** 2 + b_J ** 2 + b_K ** 2)
        bprh_loss = f_Theta - regula

        # get derivatives and update

        # NABULA U_u
        df_dUu = sigmoid(- r_hat_uIJ / spec_alpha_u) / spec_alpha_u * (V_bar_I - V_bar_J) + sigmoid(- r_hat_uJK) * (V_bar_J - V_bar_K)
        dR_dUu = 2 * lambda_u * U_u
        # update U_u = U_u + gamma * (df_dUu - dR_dUu)
        U[u_index, :-1] += gamma * (df_dUu - dR_dUu)

        # NABULA V_i
        df_dbi = sigmoid(- r_hat_uIJ / spec_alpha_u) / (len(I) * spec_alpha_u)
        dR_dbi = 2 * lambda_b * b_I / len(I)
        df_dVi = df_dbi * U_u
        dR_dVi = 2 * lambda_v * V_bar_I / len(I)
        # update V_i = V_i + gamma * (df_dVi - dR_dVi)
        V[:-1, adv_index(item_list, sorted(I))] += gamma * (df_dVi - dR_dVi)[:,None] # trick: transpose here
        # update b_i = b_i + gamma * (df_dbi - dR_dbi)
        V[-1, adv_index(item_list, sorted(I))] += gamma * (df_dbi - dR_dbi)

        # NABULA V_j
        df_dbj = (- sigmoid(- spec_alpha_u * r_hat_uIJ) / spec_alpha_u + sigmoid(- r_hat_uJK)) / len(J)
        dR_dbj = 2 * lambda_b * b_J / len(J)
        df_dVj = df_dbj * U_u
        dR_dVj = 2 * lambda_v * V_bar_J / len(J)
        # update V_j = V_j + gamma * (df_dVj - dR_dVj)
        V[:-1, adv_index(item_list, sorted(J))] += gamma * (df_dVj - dR_dVj)[:,None] # trick: transpose here
        # update b_j = b_j + gamma * (df_dbj - dR_dbj)
        V[-1, adv_index(item_list, sorted(J))] += gamma * (df_dbj - dR_dbj)

        # NABULA V_k
        df_dbk = - sigmoid(- r_hat_uJK) / len(K)
        dR_dbk = 2 * lambda_b * b_K / len(K)
        df_dVk = df_dbk * U_u
        dR_dVk = 2 * lambda_v * V_bar_K / len(K)
        # update V_k = V_k + gamma * (df_dVk - dR_dVk)
        V[:-1, adv_index(item_list, sorted(K))] += gamma * (df_dVk - dR_dVk)[:,None] # trick: transpose here
        # update b_k = b_k + gamma * (df_dbk - dR_dbk)
        V[-1, adv_index(item_list, sorted(K))] += gamma * (df_dbk - dR_dbk)

        # update estimation
        estimation = np.dot(U, V)
        # Postfix will be displayed on the right,
        # formatted automatically based on argument's datatype
        t.set_postfix(loss=bprh_loss, u=u, i=i, j=j, k=k, I=I, J=J, K=K)

ITER 2499: 100%|██████████| 2500/2500 [00:08<00:00, 281.09it/s, I={'I2'}, J={'I1', 'I3'}, K={'I4'}, i=I2, j=I3, k=I4, loss=-0.832, u=U5]
